Etape 1: Creation du DataSet

In [17]:
import pandas 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler


#df_3features=pandas.read_csv("./datasets/Stress_temperature_step_humidity.csv", sep =',')
df=pandas.read_csv("./datasets/Sleep_health.csv", sep =',')
#df_sleep=pandas.read_csv("./datasets/sayopillow/SaYoPillow.csv", sep =',')


In [36]:

def hot_encode_sleep_disorder(df):
    df['Sleep Disorder'] = df['Sleep Disorder'].map({'Sleep Apnea' : 1, 'Insomnia': 2})
    feature_to_encode = df[['Sleep Disorder']]

    # Créez une instance de OneHotEncoder
    one_hot_encoder = OneHotEncoder(sparse=False)

    # Ajustez et transformez les données
    one_hot_encoded = one_hot_encoder.fit_transform(feature_to_encode)

    # Créez un DataFrame avec les nouvelles colonnes
    columns_names = ['Normal_sleep', 'Sleep_apnea', 'Insomnia']
    one_hot_df = pandas.DataFrame(one_hot_encoded, columns=columns_names)

    # Concaténez le DataFrame one-hot avec le DataFrame d'origine
    df = pandas.concat([df, one_hot_df], axis=1)

    # Supprimez la colonne 'sommeil' et 'Normal_sleep' considéré comme inutile
    df = df.drop(['Sleep Disorder'], axis=1)
    df = df.drop(['Normal_sleep'], axis=1)

In [35]:
def sleep_disorder(df):
    df['Sleep Disorder'] = df['Sleep Disorder'].map({'Sleep Apnea' : True, 'Insomnia': True})

In [4]:

#6 features 
print(df["Sleep Disorder"].unique())
print(df["Sleep Duration"].unique())
print(df["Quality of Sleep"].unique())
print(df["Stress Level"].unique())
print(df["Physical Activity Level"].unique())
print(df["Heart Rate"].unique())
print(df["Daily Steps"].unique())

[nan 'Sleep Apnea' 'Insomnia']
[6.1 6.2 5.9 6.3 7.8 6.  6.5 7.6 7.7 7.9 6.4 7.5 7.2 5.8 6.7 7.3 7.4 7.1
 6.6 6.9 8.  6.8 8.1 8.3 8.5 8.4 8.2]
[6 4 7 5 8 9]
[6 8 7 4 3 5]
[42 60 30 40 75 35 45 50 32 70 80 55 90 47 65 85]
[77 75 85 82 70 80 78 69 72 68 76 81 65 84 74 67 73 83 86]
[ 4200 10000  3000  3500  8000  4000  4100  6800  5000  7000  5500  5200
  5600  3300  4800  7500  7300  6200  6000  3700]


### 1. DESCRIPTION DES FEATURES ###  

**Features déjà presentes dans le dataset initial**   
Gender : Male or Female (**Initial Data from individual**)  
Age : Age of the induvidual (**Initial Data from individual**)  
Sleep Duration : in hours decimal (**Watch**)  
Quality of Sleep : Assessed on a 0-10 scale (**assessed by questionnaire (or watch**))   
Physical Activity Level : number of minutes spent exercising during the day (**watch or questionnaire**)  
Heart Rate : BPM nominale (**Initial Data from individual (or rate mesured during the night by watch)**)  
Daily Steps : number of steps during the day (**watch**)  
Sleep Disorder : [Regular, Sleep Apnea, Insomnia'] (**Initial Data from individual**)  

**Features à ajouter dans le dataset**  
irritation level : Assessed on a 0-10 scale (**assessed by questionnaire**)  
Control level : Assessed on a 0-10 scale (**assessed by questionnaire**)  
Emotion Control level : Assessed on a 0-10 scale (**assessed by questionnaire**)
Heart mean day : mean of BPM during the day (**watch**)   

**Target**  
Stress Level : Target feature (1 - not stressed, 2 - regular, 3 - stressed)    

### 2. Transformation des données du dataset initial ###  
- Utilisation d'un pipeline

In [37]:
# Effectue des transformations sur 'df' qui doit contenir les features initiales
def data_transform(df):
    
    df.fillna(False, inplace=True)
    #initialement les valeurs des features sont entre 0 et 10
    df.drop(["Person ID","Occupation","Blood Pressure"], axis = 1, inplace=True)
    df.drop( labels="BMI Category", axis = 1, inplace = True)
    
    df['Gender'] = df['Gender'].map({'Male': 0, 'Female': 1})

    #hot_encode_sleep_disorder(df)
    sleep_disorder(df)
    df.fillna(False, inplace=True)
    
    df['Irritation Level'] =(( df['Stress Level'] + (10 - df['Quality of Sleep']))  
                                // 2 + np.random.choice([-2, -1, 0, 1, 2], len(df)))
    df['Irritation Level'] = np.clip(df['Irritation Level'], 0, 10) #on s'assure que ça reste entre 0 et 10

    df['Life Control Level'] =( (10 - df['Stress Level'])
                       + np.random.choice([-2, -1, 0, 1, 2], len(df)))
    df['Life Control Level'] = np.clip(df['Life Control Level'], 0, 10) #on s'assure que ça reste entre 0 et 10
    
    df['Emotion Control Level'] =(( (df['Physical Activity Level'] / 9)  + 
                                          (10 - df['Stress Level']))  // 2 + 
                                          np.random.choice([-2, -1, 0, 1, 2], len(df)))
    df['Emotion Control Level'] = np.clip(df['Emotion Control Level'], 0, 10) #on s'assure que ça reste entre 0 et 10

    df['Mean Heart Rate'] = ( df['Heart Rate'] +
    np.maximum(5, ((df['Stress Level'] - 5) + ((df['Physical Activity Level']) * 5 / df['Physical Activity Level'].max()))) +
    np.random.choice([-2, -1, 0, 1, 2], len(df)))

    df['BPM diff'] = df['Mean Heart Rate'] - df['Heart Rate'] 
    df.drop('Mean Heart Rate', axis = 1, inplace=True)
    df.drop('Heart Rate', axis = 1, inplace=True)



    


In [53]:

#df_health.to_csv('Premier_DataFrama.csv', index=False)

In [38]:
def scale_data(df):
    scaler = MinMaxScaler()

    # Choix des colonnes à "scaler"
    columns_to_scale = [col for col in df.columns if col != 'Stress Level']

    # Ajustement du scaler sur les données
    scaler.fit(df[columns_to_scale])

    # Transformation des colonnes sélectionnées entre 0 et 1
    df[columns_to_scale] = scaler.transform(df[columns_to_scale])

In [39]:
df=pandas.read_csv("./datasets/Sleep_health.csv", sep =',')
data_transform(df)
scale_data(df)
df.iloc[:10]

,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Daily Steps,Sleep Disorder,Irritation Level,Life Control Level,Emotion Control Level,BPM diff
0,0.0,0.00000,0.111111,0.4,0.200000,6,0.171429,0.0,0.555556,0.666667,0.666667,0.142857
1,0.0,0.03125,0.148148,0.4,0.500000,8,1.000000,0.0,0.555556,0.000000,0.666667,0.619048
2,0.0,0.03125,0.148148,0.4,0.500000,8,1.000000,0.0,0.444444,0.111111,0.222222,0.190476
3,0.0,0.03125,0.037037,0.0,0.000000,8,0.000000,1.0,0.666667,0.000000,0.333333,0.000000
4,0.0,0.03125,0.037037,0.0,0.000000,8,0.000000,1.0,0.888889,0.111111,0.000000,0.428571
5,0.0,0.03125,0.037037,0.0,0.000000,8,0.000000,1.0,0.666667,0.444444,0.000000,0.285714
6,0.0,0.06250,0.185185,0.4,0.166667,7,0.071429,1.0,0.666667,0.222222,0.555556,0.285714
7,0.0,0.06250,0.740741,0.6,0.750000,6,0.714286,0.0,0.333333,0.555556,0.888889,0.595238
8,0.0,0.06250,0.740741,0.6,0.750000,6,0.714286,0.0,0.555556,0.333333,0.666667,0.452381
9,0.0,0.06250,0.740741,0.6,0.750000,6,0.714286,0.0,0.333333,0.444444,0.888889,0.595238


Santé du cœur:

Fréquence cardiaque de jour
Fréquence cardiaque de nuit
Notifications de fréquence cardiaque élevée et basse
Variations de fréquence cardiaque de nuit*
*Fonctionnalité attendue pour Q4 2023

Suivi d’activité:

Zones de fréquence cardiaque en entraînement
Minutes actives
Nombre de pas
Calories brûlées
Connexion GPS : distance, allure & dénivelé

Suivi du sommeil:

Score de qualité du sommeil
Phases de sommeil (profond, léger)
Fréquence respiratoire*
Perturbations respiratoires
Durée du sommeil
Interruptions
Régularité